In [ ]:
import socket
import sys
import time

In [ ]:
device_port = 61499

In [ ]:
class DeviceConnection():
    
    def __init__(self, port):
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.socket.connect(('localhost', port))
        self.id = 0
   
    def send(self, resource, action, body):
        content = '<Request ID="' + str(self.id) + '" Action="' + action + '">' + body + '</Request>'
        self.id = self.id + 1
        encoded = ('\x50\x00'+ chr(len(resource)) + resource + '\x50\x00' + chr(len(content)) + content).encode('ascii')
        self.socket.sendall(encoded)
        print(self.socket.recv(1024)[3:].decode('ascii'))
    
    def close(self):
        self.socket.close()
    

In [ ]:
connection = DeviceConnection(device_port)

In [ ]:
connection.send('', 'CREATE', '<FB Name="Blinky_RES" Type="EMB_RES" />')

In [ ]:
connection.send('Blinky_RES', 'CREATE', '<FB Name="E_CYCLE" Type="E_CYCLE" />')
connection.send('Blinky_RES', 'CREATE', '<FB Name="E_SWITCH" Type="E_SWITCH" />')
connection.send('Blinky_RES', 'CREATE', '<FB Name="E_SR" Type="E_SR" />')
connection.send('Blinky_RES', 'CREATE', '<Connection Source="START.COLD" Type="E_CYCLE.START" />')
connection.send('Blinky_RES', 'CREATE', '<Connection Source="START.WARM" Type="E_CYCLE.START" />')
connection.send('Blinky_RES', 'CREATE', '<Connection Source="E_CYCLE.EO" Type="E_SWITCH.EI" />')
connection.send('Blinky_RES', 'CREATE', '<Connection Source="E_SWITCH.EO0" Type="E_SR.S" />')
connection.send('Blinky_RES', 'CREATE', '<Connection Source="E_SWITCH.EO1" Type="E_SR.R" />')
connection.send('Blinky_RES', 'WRITE', '<Connection Source="T#1s" Type="E_CYCLE.DT" />')
connection.send('Blinky_RES', 'CREATE', '<Connection Source="E_SR.Q" Type="E_SWITCH.G" />')

In [ ]:
connection.send('Blinky_RES', 'START', '')

In [ ]:
connection.send('Blinky_RES', 'CREATE', '<Watch Source="E_SWITCH.EO0" Destination="*" />')
connection.send('Blinky_RES', 'CREATE', '<Watch Source="E_CYCLE.EO" Destination="*" />')
connection.send('Blinky_RES', 'CREATE', '<Watch Source="E_CYCLE.E_DELAY.EO" Destination="*" />')
connection.send('Blinky_RES', 'CREATE', '<Watch Source="E_SR.Q" Destination="*" />')

In [ ]:
for i in range(100):
    connection.send('', 'READ', '<Watches/>')
    time.sleep(0.1)

In [ ]:
connection.close()

In [ ]:
s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
s.bind(('239.0.0.1', 61000))

data = s.recvfrom(16)
data

In [ ]:
import struct

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)
sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
sock.bind(('', 61000))

mreq = struct.pack("=4sl", socket.inet_aton('239.0.0.1'), socket.INADDR_ANY)
sock.setsockopt(socket.IPPROTO_IP, socket.IP_ADD_MEMBERSHIP, mreq)

while True:
    print([i for i in sock.recv(10240)])